In [1]:
import pyodbc
import pandas
from sqlalchemy import create_engine
from cx_Oracle import DatabaseError
import numpy as np
import io, csv, time

In [2]:
#Connect to the 
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=CanadianBusinessPoints1701BC.mdb;'
    )
conn = pyodbc.connect(conn_str)
sql = """ SELECT * FROM "CanadianBusinessPoints1701_BC" """
data = pandas.read_sql(sql,conn).to_csv('CanadianBusinessPoints1701BC.csv')
conn.close()

In [6]:
# Create Oracle connection and cursor
address = ''
engine = create_engine(address)
connection = engine.raw_connection()
cursor = connection.cursor()

#create the table but first drop if it already exists
try:
    command = """DROP TABLE src_pb_businesspoints"""
    cursor.execute(command)
    print('Dropped Table src_pb_businesspoints')
except DatabaseError as e:
    print(e)
    
command = """
    CREATE TABLE src_pb_businesspoints(
      id NUMBER GENERATED BY DEFAULT AS IDENTITY,
      name VARCHAR2(30), 
      city VARCHAR(30), 
      prov VARCHAR(2), 
      fsaldu VARCHAR(6), 
      sic VARCHAR(6), 
      frncod char(6), 
      year VARCHAR2(4), 
      ttlcd VARCHAR(1), 
      gendcd VARCHAR(1), 
      empsiz VARCHAR(1), 
      salvol VARCHAR(1), 
      iscode VARCHAR2(1), 
      locnum VARCHAR2(9), 
      prmsic VARCHAR2(6), 
      ssic1 VARCHAR2(6),
      ssic2 VARCHAR2(6),
      ssic3 VARCHAR2(6),
      ssic4 VARCHAR2(6),
      scaddr VARCHAR2(30),
      sccity VARCHAR2(30),
      scprov VARCHAR2(2),
      scpcd VARCHAR2(7),
      frflag VARCHAR2(1),
      hdbrch VARCHAR2(1),
      subnum VARCHAR2(9),
      ultnum VARCHAR2(9),
      locemp INTEGER,
      pactem INTEGER,
      psalvl VARCHAR2(1),
      latitude NUMBER(20,6),
      longitude NUMBER(20,6),
      can VARCHAR2(2),
      pr VARCHAR2(2),
      prcd VARCHAR2(4),
      prcdcsd VARCHAR2(7),
      cma VARCHAR2(3),
      cmact VARCHAR2(10),
      fsa VARCHAR2(3),
      prcdda VARCHAR2(8),
      prfed VARCHAR2(5),
      centroid VARCHAR2(2),
      descr VARCHAR2(50),
      frndsc VARCHAR2(40),
      inddes VARCHAR2(40),
      cansic VARCHAR2(4),
      empdescr VARCHAR2(12),
      saldesc VARCHAR2(25),
      naics VARCHAR2(8),
      naicsdesc VARCHAR2(50), 
      ptcode VARCHAR2(3),
      ifcode VARCHAR2(1), 
      frncode2 VARCHAR2(18), 
      site_num VARCHAR2(9),
      geom SDO_GEOMETRY,
      CONSTRAINT businesspoints_pkey PRIMARY KEY(id)
)"""

cursor.execute(command)
connection.commit()
print('Added Table src_pb_businesspoints')
#close cursor and connection
cursor.close()
connection.close()

#stream the data from pandas using 'to_csv' and StringIO(); then use cx_oracles 'execute_many' function
# This splits the DataFrame into 50 chunks to allow smaller Database operations
df_list = np.array_split(pandas.read_csv('CanadianBusinessPoints1701BC.csv',encoding='latin_1',dtype='str'),50)
frame_count=len(df_list)
i = 0

for frame in df_list:
    i += 1
    output = io.StringIO()
    
    frame = frame[['name','city','prov','fsaldu','sic','frncod','year','ttlcd','gendcd','empsiz','salvol','iscode','locnum',
                   'prmsic','ssic1','ssic2','ssic3','ssic4','scaddr','sccity','scprov','scpcd','frflag','hdbrch','subnum',
                   'ultnum','locemp','pactem','psalvl','latitude','longitude','can','pr','prcd','prcdcsd','cma','cmact','fsa',
                   'prcdda','prfed','centroid','descr','frndsc','inddes','cansic','empdescr','saldesc','naics','naicsdesc',
                   'ptcode','ifcode','frncode2','site_num']]
    
    frame.to_csv(output, sep=',', index=False,encoding='latin_1')
    output.seek(0)
    reader = csv.DictReader(output)
    rows = [row for row in reader]
        
    command = """
        INSERT INTO 
            src_pb_businesspoints(name,city,prov,fsaldu,sic,frncod,year,ttlcd,gendcd,empsiz,salvol,iscode,locnum,
             prmsic,ssic1,ssic2,ssic3,ssic4,scaddr,sccity,scprov,scpcd,frflag,hdbrch,subnum,ultnum,locemp,pactem,
             psalvl,latitude,longitude,can,pr,prcd,prcdcsd,cma,cmact,fsa,prcdda,prfed,centroid,descr,frndsc,inddes,
             cansic,empdescr,saldesc,naics,naicsdesc,ptcode,ifcode,frncode2,site_num,geom)
        VALUES (:name,:city,:prov,:fsaldu,:sic,:frncod,:year,:ttlcd,:gendcd,:empsiz,:salvol,:iscode,:locnum,
            :prmsic,:ssic1,:ssic2,:ssic3,:ssic4,:scaddr,:sccity,:scprov,:scpcd,:frflag,:hdbrch,:subnum,:ultnum,:locemp,
            :pactem,:psalvl,:latitude,:longitude,:can,:pr,:prcd,:prcdcsd,:cma,:cmact,:fsa,:prcdda,:prfed,:centroid,:descr,
            :frndsc,:inddes,:cansic,:empdescr,:saldesc,:naics,:naicsdesc,:ptcode,:ifcode,:frncode2,:site_num,
            SDO_CS.TRANSFORM(SDO_GEOMETRY(2001,4326,SDO_POINT_TYPE(:longitude,:latitude, NULL),NULL,NULL),3005))
    """
    
    commited = False
    while commited == False:
        try:
            # Create Oracle connection and cursor
            connection = engine.raw_connection()
            cursor = connection.cursor()
            cursor.prepare(command)
            cursor.executemany(None, rows)
            connection.commit()
            commited = True
        except Exception as e:
            print(e)
            print('Trying Again in 10 seconds')
            time.sleep(10)
                               
    print('Committed frame {0} of {1}'.format(i,frame_count))

    #close cursor and connection
    cursor.close()
    connection.close()

connection = engine.raw_connection()
cursor = connection.cursor()

# Drop and Add the table to the user_sdo_metadata table
try:
    command = """DELETE FROM user_sdo_geom_metadata WHERE table_name = 'SRC_PB_BUSINESSPOINTS'"""
    cursor.execute(command)
    print('Removed SRC_PB_BUSINESSPOINTS from user_sdo_geom_metadata')
except DatabaseError as e:
    print(e)

command = """
    INSERT INTO
      user_sdo_geom_metadata (
        table_name,
        column_name,
        diminfo,
        srid
      ) VALUES (
        'SRC_PB_BUSINESSPOINTS',
        'GEOM', 
        sdo_dim_array(
          sdo_dim_element('X',395388.99,1854357.76,0.005),
          sdo_dim_element('Y',369155.12,1714756.85,0.005)
        ), 
        3005
      )
    """
cursor.execute(command)
connection.commit()
print('Added SRC_PB_BUSINESSPOINTS to user_sdo_geom_metadata')
# Drop and add the spatial index
try:
    command = """DROP INDEX businesspoints_spidx"""
    cursor.execute(command)
    connection.commit()
    print('Dropped Spatial Index businesspoints_spidx')
except DatabaseError as e:
    print(e)
    
command = """
      CREATE INDEX
        businesspoints_spidx 
      ON 
        src_pb_businesspoints(geom)
      INDEXTYPE IS 
        MDSYS.SPATIAL_INDEX
"""
cursor.execute(command)
connection.commit()
print('Added Spatial Index businesspoints_spidx')
#close cursor and connection
cursor.close()
connection.close()
print('Finished')

Dropped Table src_pb_businesspoints
Added Table src_pb_businesspoints
Committed frame 1 of 50
Committed frame 2 of 50
Committed frame 3 of 50
Committed frame 4 of 50
Committed frame 5 of 50
Committed frame 6 of 50
Committed frame 7 of 50
Committed frame 8 of 50
Committed frame 9 of 50
Committed frame 10 of 50
Committed frame 11 of 50
Committed frame 12 of 50
Committed frame 13 of 50
Committed frame 14 of 50
Committed frame 15 of 50
Committed frame 16 of 50
Committed frame 17 of 50
Committed frame 18 of 50
Committed frame 19 of 50
Committed frame 20 of 50
Committed frame 21 of 50
Committed frame 22 of 50
Committed frame 23 of 50
Committed frame 24 of 50
Committed frame 25 of 50
Committed frame 26 of 50
Committed frame 27 of 50
Committed frame 28 of 50
Committed frame 29 of 50
Committed frame 30 of 50
Committed frame 31 of 50
Committed frame 32 of 50
Committed frame 33 of 50
Committed frame 34 of 50
Committed frame 35 of 50
Committed frame 36 of 50
Committed frame 37 of 50
Committed fram